### Extraccion de datos ###

In [1]:
## EXTRACCION DE DATOS

import cv2
import numpy as np
from pdf2image import convert_from_path
import pytesseract
import pandas as pd
import re

# Convertir PDF a imágenes
pages = convert_from_path('Extracto Banco Credicoop.pdf', 300)  # Utilizar el PDF con líneas
# Definir el área de interés (x1, y1, x2, y y2)
area_of_interest = {"x1": 25, "y1": 900, "x2": 3100, "y2": 3800}

# Configuración personalizada de pytesseract
custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz.-,:;/ -c preserve_interword_spaces=1 -c user_defined_dpi=300'

# Lista para almacenar los DataFrames resultantes
lista_de_dataframes = []

# Recorrer cada página convertida en imagen
for page_number, page in enumerate(pages):
    # Convertir la imagen de PIL a formato OpenCV
    page_cv = np.array(page)
    page_cv = cv2.cvtColor(page_cv, cv2.COLOR_RGB2GRAY)  # Convertir a escala de grises

    # Extraer las coordenadas del área de interés y recortar
    x1, y1, x2, y2 = area_of_interest["x1"], area_of_interest["y1"], area_of_interest["x2"], area_of_interest["y2"]
    cropped = page_cv[y1:y2, x1:x2]

    # Preprocesar la imagen
    _, binary = cv2.threshold(cropped, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  # Binarización
    denoised = cv2.fastNlMeansDenoising(binary, None, 30, 7, 21)  # Eliminación de ruido
    contrast = cv2.convertScaleAbs(denoised, alpha=1.5, beta=0)  # Aumento de contraste

    # Extraer texto del área preprocesada
    data = pytesseract.image_to_data(contrast, output_type=pytesseract.Output.DICT, lang="spa", config=custom_config)

    # Convertir el diccionario resultante en un DataFrame y agregarlo a la lista
    df_temp = pd.DataFrame(data)
    df_temp['Page'] = page_number + 1  # Añadir columna de página
    lista_de_dataframes.append(df_temp)

# Concatenar todos los DataFrames en uno solo
df_pytessect = pd.concat(lista_de_dataframes, ignore_index=True)


### Limpieza de datos ###

In [2]:
def filtrar_por_pagina(df, pagina):
    return df[df['Page'] == pagina]  # Filtrar el DataFrame por el número de página.

def cortar_desde_hasta(df, start_text, end_text):
    # Verificar si existen las palabras clave en el DataFrame
    if not df['text'].str.contains(start_text).any():
        return pd.DataFrame()  # Retornar un DataFrame vacío si no existe 'start_text'
    start_index = df[df['text'].str.contains(start_text)].index[0]  # Índice de la primera aparición del texto de inicio
    if not df['text'].str.contains(end_text).any():
        return pd.DataFrame()  # Retornar un DataFrame vacío si no existen 'end_text'
    end_index = df[(df['text'].str.contains(end_text)) & (df.index > start_index)].index[0]  # Índice de la primera aparición del texto de fin después del inicio
    return df.loc[start_index:end_index]  # Cortar el DataFrame entre los índices encontrados

# Procesar cada página
max_page = df_pytessect['Page'].max()
lista_de_paginas = []
for page in range(1, max_page + 1):
    df_pagina = filtrar_por_pagina(df_pytessect, page)
    df_cortado = cortar_desde_hasta(df_pagina, "FECHA", "CONTINUA")
    if not df_cortado.empty:
        lista_de_paginas.append(df_cortado)

# Concatenar todos los DataFrames cortados en uno solo
df_filtrado = pd.concat(lista_de_paginas, ignore_index=True)

# Filtrar la última página
max_page2 = df_filtrado['Page'].max()-1
df_Resto = df_filtrado[df_filtrado['Page'] < max_page2]
df_Last = df_filtrado[df_filtrado['Page'] == max_page2]
df_Last_filter = cortar_desde_hasta(df_Last, "FECHA", "DENOMINACION")
df_limpio = pd.concat([df_Resto, df_Last_filter], ignore_index=True)
df_table = df_limpio[df_limpio['text'] != ""][:-1]
df_table = df_table[~df_table['text'].str.contains("CONTINUA")]

# Calcular las coordenadas absolutas
df_table['Esq Sup Izq x1'] = df_table['left'] + area_of_interest['x1']

# Clasificar las columnas por coordenadas
def calcular_valor(x):
    if x > 2650:
        return 6
    elif x > 2150:
        return 5
    elif x > 1700:
        return 4
    else:
        return 0

df_table['Column'] = df_table['Esq Sup Izq x1'].apply(calcular_valor)
df_concatenado = df_table.groupby(['Page', 'par_num', 'line_num', 'Column'], as_index=False).agg({'text': ' '.join})
df_pivoted = df_concatenado.pivot(index=['Page', 'par_num', 'line_num'], columns='Column', values='text')
df_pivoted.reset_index(inplace=True)

# Dividir la columna 'text' en 'Fecha' y 'Resto del Texto'
def split_text(row):
    if re.match(r'\d{2}/\d{2}/\d{2}', row[0][:8]):
        return pd.Series([row[0][:8], row[0][8:].strip()])
    else:
        return pd.Series(['', row[0]])

df_pivoted[['Fecha', 'Resto del Texto']] = df_pivoted.apply(split_text, axis=1)

# Separar números y texto en 'Resto del Texto'
def separar_numeros_texto(row):
    if row['Fecha']:
        match = re.match(r"(\d+)(.*)", row['Resto del Texto'])
        if match:
            return pd.Series([match.group(1), match.group(2).strip()])
        else:
            return pd.Series(['', row['Resto del Texto']])
    else:
        return pd.Series(['', row['Resto del Texto']])

df_pivoted[['Comprobante', 'Descripcion']] = df_pivoted.apply(separar_numeros_texto, axis=1)
df_banco = df_pivoted.drop(columns=['par_num', 'line_num', 0, "Resto del Texto"])

# Insertar espacios en la columna 'Descripcion'
def insertar_espacios(texto):
    texto = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', texto)
    texto = re.sub(r'(?<=[a-zA-Z])(?=\d)', ' ', texto)
    texto = re.sub(r'(?<=\d)(?=[a-zA-Z])', ' ', texto)
    texto = re.sub(r'(\d{2}/\d{2})(\d{2}:\d{2})', r'\1 \2', texto)
    return texto

df_banco['Descripcion'] = df_banco['Descripcion'].apply(insertar_espacios)
df_banco = df_banco[~df_banco['Descripcion'].str.contains("FECHA")]
df_banco.rename(columns={4: 'Debitos', 5: 'Creditos', 6: 'Saldo'}, inplace=True)
df_banco = df_banco[['Page', 'Fecha', 'Comprobante', 'Descripcion', 'Debitos', 'Creditos', 'Saldo']]


### Extraccion de Informacion Adicional 

In [3]:
# Convertir la primera página del PDF a imagen
page0 = pages[0]
page_cv0 = np.array(page0)
page_cv0 = cv2.cvtColor(page_cv0, cv2.COLOR_RGB2GRAY)

# Recortar el área de interés y preprocesar la imagen
x0_1, y0_1, x0_2, y0_2 = 35, 600, 3000, 1400
cropped0 = page_cv0[y0_1:y0_2, x0_1:x0_2]
_, binary0 = cv2.threshold(cropped0, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
denoised0 = cv2.fastNlMeansDenoising(binary0, None, 30, 7, 21)
contrast0 = cv2.convertScaleAbs(denoised0, alpha=1.5, beta=0)
pagina0 = pytesseract.image_to_data(contrast0, output_type=pytesseract.Output.DICT, lang="spa", config=custom_config)

# Extraer CBU y banco
col_text = pd.DataFrame(pagina0['text'])
CBU = col_text.iloc[-1][0][-22:]  # Extraer los últimos 22 caracteres como CBU
banco = col_text[col_text[0].str.contains('CUIT')].iloc[0]
banco = "Banco " + banco[0][-13:]  # Formatear el nombre del banco

# Extraer y formatear el periodo
Periodo_text = col_text[col_text[0].str.contains('Cta')].iloc[0][0]
fechas = re.findall(r'\d{2}/\d{2}/\d{4}', Periodo_text)
fecha1 = '-'.join(fechas[0].split('/')[::-1])
fecha2 = '-'.join(fechas[1].split('/')[::-1])
periodo = f"{fecha2} a {fecha1}"

# Crear nombre del archivo Excel basado en CBU y periodo
name_excel = f"{banco}_CBU {CBU}_{periodo}.csv"

# Guardar el DataFrame en un archivo Excel
df_banco.to_csv(name_excel, sep=';', encoding='latin-1', index=False)
print(f"DataFrame guardado en '{name_excel}'")

DataFrame guardado en 'Banco 30-57142135-2_CBU 1910142455014251145146_2019-03-31 a 2018-10-01.csv'
